In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.manifold import TSNE
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.manifold import Isomap
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn import metrics
from itertools import cycle


import time
import logging

In [ ]:
indx = [ str(i) for i in range(1,785) ]

df = pd.read_csv("/home/yekta/Desktop/prjct/train.csv", sep=',')

#np.random.seed(42)
#randm = np.random.permutation(df.shape[0])
#df_subset = df.loc[randm[:10000],:].copy()

data=df[indx]
label=df['Label']

del df

scaler = preprocessing.StandardScaler(with_mean=True, with_std=True)

#Either use 0.8 ratio or use the full data set with 10-fold cross validation
ran = 42
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, random_state = ran)
del data
del label

X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [ ]:
indx2 = [ str(i) for i in range(0,784) ]
df_test = pd.read_csv("/home/yekta/Desktop/prjct/testX.csv", sep=',')
X_test=df_test[indx2]
del df_test

start_time = time.time()      
pca = PCA()
pca_data=pca.fit_transform(X_train)
pca_data=pca_data[:,:90]
print(time.time() - start_time)


X_test=pca.transform(X_test)
X_test=X_test[:,:90]

start_time = time.time()      
classifier = SVC(kernel='rbf', C=10, random_state=42)    
classifier.fit(pca_data, y_train)
print(time.time() - start_time)

start_time = time.time()      
y_pred = classifier.predict(X_test)
print(time.time() - start_time)

y_te = label_binarize(y_test, classes=[0, 1, 2, 3, 4])
y_pre = label_binarize(y_pred, classes=[0, 1, 2, 3, 4])


############################################################
#ROC curve
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(5):
    fpr[i], tpr[i], _ = roc_curve(y_te[:, i], y_pre[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

lw=2
colors = cycle(['aqua', 'darkorange', 'cornflowerblue','red','sienna'])
for i, color in zip(range(5), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()


indx3=[i for i in range(0,10000)]
save=pd.DataFrame(indx3,columns=['Id'])
save['Label']=y_pred
save.to_csv("predicted.csv", index=False)

###########################################################################
#Parameter optimization
#listo=[8,9,11,12]
#for i in listo: 
    classifier = SVC(kernel='rbf', C=11, random_state=42)    
    start_time = time.time()      
    fins=cross_val_score(classifier,pca_data,label, cv=10)
    print(str(i)+" : "+str(fins.mean())+" time is: "+str(time.time() - start_time)+"\n")
    logging.debug(str(i)+" : "+str(fins.mean())+" time is: "+str(time.time() - start_time)+"\n")
    print(i)
    
#lda = LDA()
#lda_data=lda.fit_transform(pca_data,label)
#del pca_data

#iso=Isomap(n_neighbors=10,n_components=10)
#lda_data = iso.fit_transform(lda_data)

#X_test=lda.transform()
#fig, axs = plt.subplots(1,3)
#fig.set_size_inches(6.4*3, 4.8)
#x=0
#y=0
#for i in range(0,3):
#    sns.scatterplot(x=lda_data[:,i], y=lda_data[:,i+1],hue=label,ax=axs[i],palette='colorblind')
#    axs[x].set_title('PC'+str(i+1)+" vs PC"+str(i+2))
#    x=x+1


#classifier = KNeighborsClassifier(n_neighbors=100, weights='distance', metric='manhattan')
#classifier.fit(X_train, y_train)
#y_pred = classifier.predict(X_test)
#accscore = accuracy_score(y_test, y_pred)




#save=pd.DataFrame(indx2,columns=['Id'])
#save['Label']=y_pred
#save.to_csv("/home/yekta/Desktop/prjct/predicted.csv", index=False)